In [17]:
import os
import sys
import time
import random
import numpy as np
import plotly.express as px

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from game_simulation import CoinGameSimulation

Cheaters are only biased against heads!

In [18]:
def simulate_strategy(strategy, n_simulations=500):
    # Simulate the game with the given strategy
    # strategy is a function that takes n_heads, n_tails and flips_left and returns an action in ("fair", "cheater", "one_flip", "five_flips")
    g = CoinGameSimulation()
    scores = []
    for _ in range(n_simulations):
        g.reset_game()
        while not g.game_over:
            data = g.get_data()
            n_heads, n_tails, flips_left = data['heads'], data['tails'], data['flips_left']
            action = strategy(n_heads, n_tails, flips_left)
            if action == "fair":
                g.label_fair()
            elif action == "cheater":
                g.label_cheater()
            elif action == "one_flip":
                g.flip_one_coin()
            elif action == "five_flips":
                g.flip_five_coins()
            else:
                raise ValueError("Unknown action: {}".format(action))
        scores.append(g.get_score())
    print(np.mean(scores), "±", np.std(scores))
    return scores

Random choice gives a score of around 7

In [16]:
_ = simulate_strategy(lambda n_heads, n_tails, flips_left: random.choice(["cheater", "fair"]))

7.166 ± 7.425257167263636


In [42]:
def maximum_likelihood(n_heads, n_tails, flips_left):
    # Return the action that maximizes the likelihood of the ground truth label
    diff = n_heads - n_tails
    num_flips = n_heads + n_tails
    if num_flips == 0:
        return "one_flip"
  
    if n_heads > n_tails:
        return "fair"
    

    if num_flips < 4:
        return "one_flip"
    else:
        return "fair"

_ = simulate_strategy(maximum_likelihood, n_simulations=10)

KeyboardInterrupt: 